Getting and organizing data

In [ ]:
#import data
import pandas as pd
youTubeData = pd.read_csv('USvideos.csv', index_col=0)
pornData = pd.read_csv('xhamster.csv', index_col=0)


In [ ]:
youTubeData

In [ ]:
#add porn column
youTubeData["isPorn"]=pd.Series()
youTubeData=youTubeData.fillna(0)


In [ ]:
#get rid of unnecessary titles
youTubeTitles = youTubeData.loc[:,["title","isPorn"]]

youTubeTitles

In [ ]:
#same as above
pornData["isPorn"]=pd.Series()
pornData=pornData.fillna(1)
pornData

In [ ]:
#same as above
pornTitles= pornData.loc[:,["title","isPorn"]]
pornTitles

In [ ]:
#merge datasets
merged_data = youTubeTitles.append(pornTitles, ignore_index=True)
title_matrix= merged_data.loc[:,["title"]]
target_array=merged_data.loc[:, ["isPorn"]]

Encode words in titles as integers

In [ ]:
#download dictionary of most frequent english words
path="../myBigData/myPorn"
with open("google-10000-english-usa.txt") as f:
    text=f.read()
    commonDict = {}
    counter=0
    for line in f:
        commonDict[counter]=line
        counter=counter+1
print commonDict

Building model <br>
Link to article I used: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [ ]:
#folding datasets for training
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, shuffle=True)
kf.get_n_splits(merged_data)
print kf

In [ ]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

iteration = 1
train = []
test = []
for train_index, test_index in kf.split(title_matrix):
    
    #Folding steps
    start_train= train_index[0]
    end_train=train_index[-1]
    start_test=test_index[0]
    end_test=test_index[-1]
    #print "TRAIN:", train_index, "TEST", test_index
    X_train, X_test = title_matrix.loc[start_train:end_train], title_matrix.loc[start_test:end_test]
    y_train, y_test = target_array.loc[start_train:end_train], target_array.loc[start_test:end_test]
    print y_train
    
    
    # fix random seed for reproducibility
    numpy.random.seed(7)

    # create the model
    total_length=len(X_train) #this needs to be the length of the dictionary we use to encode the titles
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(total_length, embedding_vecor_length, input_length=None))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train, y_train, nb_epoch=3, batch_size=64)

    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

    
    #RNN model above

    